In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "Vikhrmodels/QVikhr-2.5-1.5B-Instruct-r"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
SYSTEM_PROMPT = "Отвечай коротко и без дополнительных служебных сообщений."
input_text = "Что такое черная жемчужина джека воробья?"

In [ ]:
messages = [
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {"role": "system", "content": "Вы — Vikhr, ИИ помощник, созданный компанией Vikhr models для предоставления полезной, честной и безопасной информации."},
    {"role": "user", "content": input_text},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    truncation=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

In [ ]:
model = model.to('cuda')
input_ids = input_ids.to('cuda')

In [ ]:
output = model.generate(
    input_ids,
    max_length=512,
    temperature=0.4,
)

In [ ]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
clean_text = re.sub(r'^(system|assistant).*\n?', '', generated_text, flags=re.MULTILINE).strip()
user_message_start = clean_text.find("user")
if user_message_start != -1:
    clean_text = clean_text[user_message_start:]
clean_text = re.sub(r'^(user|assistant)\n', '', clean_text, flags=re.MULTILINE)
clean_text = '\n'.join(line for line in clean_text.split('\n') if line.strip())

print(clean_text)